In [2]:
import random
import networkx  
import pgmpy.models as pgmm  
import pgmpy.factors.discrete as pgmf              
import pgmpy.inference as pgmi
from random import randint

In [3]:
#Entrada de datos del buscaminas
filas = 6
columnas = 6
minas = 6

In [ ]:
# Ahora, debemos de crear el tablero del juego, el cual dependerá del numero de filas, columnas y minas
def tablero_buscaminas (filas, columnas, minas):
    # Antes de nada, se debe de comprobar que el número de minas existente en el tablero sea menor o igual al número de casillas
    if(minas > filas * columnas):
        raise Exception ("El número de minas debe ser menor o igual a" + filas*columnas)
        
    # A continuación se pasa a construir el tablero
    tableroBuscaminas = [[0 for y in range(columnas)] for x in range(filas)]
    
    # Vamos a pintar las minas
    for i in range(minas):
        fila_random = -1
        columna_random = -1
        while(fila_random = -1 || columna_random = -1 || tableroBuscaminas[fila_random][columna_random] == 'X'):
            fila_random = randint(0, filas -1)
            columna_random = randint(0, columnas-1)
        # En la fila y columna se le asigna el valor X que indica que en la casilla existe una mina
        tableroBuscaminas[fila_random][columna_random] = "X"